#Libraries requirments

In [ ]:
%%capture
!pip install requests
!pip install html5lib
!pip install bs4
!pip install fake_useragent
!pip install retrying


In [ ]:
#import packages
import pandas as pd
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import csv
import time
from tqdm.notebook import trange, tqdm
from fake_useragent import UserAgent
from retrying import retry
import glob
ua = UserAgent()
header = {
    "User-Agent": ua.random
}
from datetime import datetime

In [8]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/BBC/'

Mounted at /content/drive
/content/drive/MyDrive/BBC


In [14]:
topics = ["world", "uk", "business", "politics", "health", "education", "science_and_environment", "technology", "entertainment_and_arts"]
# create empty arrays
urls = []
titles = []
descriptions = []
dates = []
time_crawl = []

# Define a retry strategy
@retry(stop_max_attempt_number=4, wait_fixed=2500)
def make_request_get_urls(topic):
        #request to pages, pull htmls and read with  BeautifulSoup
        page = f'https://feeds.bbci.co.uk/news/{topic}/rss.xml'
        try:
          r = requests.get(page, headers=header, timeout=(4))
        except requests.exceptions.Timeout:
          print("request timeout error on Page")
          pass
        soup = BeautifulSoup(r.content, 'html5lib')
        table = soup.find('channel')


        # Extract urls from website
        for i in table.findAll("item"):
            titles.append(i.title.text.split("[CDATA[")[1].split("]]")[0])
            urls.append(i.guid.text)
            descriptions.append(str(i.description).split("[CDATA[")[1].split("]]")[0])
            dates.append(str(i.pubdate).split("<pubdate>")[1].split("</pubdate>")[0])
            time_crawl.append(datetime.now())
        df = pd.DataFrame({'title': titles,
                           'url': urls,
                           'description': descriptions,
                           'date': dates,
                           'time_crawl': time_crawl})
        # save to dataframe
        df.to_csv(f'Data/BBC_News_{topic}.csv', index=False, encoding='utf-8-sig')
        #Use time sleep to dont get error from website
        time.sleep(4)
        return df

In [ ]:
for topic in topics:
  make_request_get_urls(topic)

<ipython-input-26-48d557ebddf6>:19: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(r.content, 'html5lib')


In [25]:
@retry(stop_max_attempt_number=4, wait_fixed=2500)
def make_request_get_news(topic):
        #request to pages, pull htmls and read with  BeautifulSoup
        df = pd.read_csv(f'Data/BBC_News_{topic}.csv', encoding='utf-8-sig')
        all_texts = []
        # Extract urls from website
        for page in df['url']:
          try:
            r = requests.get(page, headers=header, timeout=(4))
          except requests.exceptions.Timeout:
            print("request timeout error on Page")
            pass
          soup = BeautifulSoup(r.content, 'html5lib')
          table = soup.find('article')
          texts = []
          # Extract urls from website
          try:
            for i in table.findAll("div", attrs = {'class' : 'sc-43e6b7ba-0 bWSguZ', 'data-component' : 'text-block'}):
                texts.append(i.text)
            all_texts.append(texts)
          except:
            all_texts.append(None)
        df['text'] = all_texts
        # save to dataframe
        df.to_csv(f'Data/main/BBC_News_{topic}.csv', index=False, encoding='utf-8-sig')
        #Use time sleep to dont get error from website
        time.sleep(4)
        return df

In [26]:
for topic in topics:
  make_request_get_news(topic)